# Package needed
## bs4
## tensorflow(or pytorch)               

In [9]:
f = open("""./2018110922gm-00a9-0000-d350bc6b.txt""", 'r', encoding='UTF8')
print(f.read())

Shuffle:
  Seed: mt19937ar-sha512-n288-base64,1rQrAHoxitgVA0XaVJkj1GViQqc4CWwdQQ260MGMbBeB2NP7zlIL3MJXlaLkT8WTVuHVku3tcSZsbmYXfyTtQsWPH75dGROKqE5/OhiSSG2YLasWH+hXqq1/SCFiJ8meOtDpvSepsRzJEaIZ5em9R/i6XgOyR7BhfUyjQ1ojgw3/hlpNTMoz3ZO3ZPC5X0YdHVCZijmm/orV6Dtm5C0JTeg4rcw92prB+jZHA7nm8K+N+s/ePoHj7Fdj0g87DHCRYXo0AJJTBgSOwOHblLrTi2TocY1GyMdxLMXMOnCsKRTu9pUbN5WmeVzXhJeiTqRF4EAw/iJBTOehysHhVce+YptKlf1UYshCTogeurEFjMG9SCed8mAFhHcoCtBUroRhMhyH3dpK0hrEq0OJhoM6QPoc3xictFqCkUVzPrWVn/CockJi2g3BWSJCLQLmMI1Z6fXTyTEytHYOL9zQe0TDYMyLidgzwfrMJwhx5zC8yGeF/Y5eef0sLqhgQUiq3N8aWYzjRfoXbqmnvuaDVNEMsLVFodE+jGPGMZlubRX9W1mqvCcImmm9MdygWe025aOG/IXv0/51E7zeFAHExQyYNGcBdkGO0PleirsOunM/IGK7dS8coYpusaZmIz3UJRddEIxC0DBY74h3wfhxpJ8pISwdqNl/eEkLYjIYCMIsBOVQax4VeocHMDRECE1FO+o8upg5TdMiZTDB/Z7FvBjNpClpmL5HtoArpspQzAveUEL6Q+n16uMo18aSK6B2qixJe1JT5g4nexbc5Qhk4vAUG5bWfBrNFAm6q1RLg8kAETxE4jIBJGxz1+dRe5Iu6hHGpPVCBxhuECt08+25RGMVM78IUKiTp1G1vW6Tz0fPmimeCCW2btKTAL9+wzJkL62pUo4b406f55l2wf7fh/NNP4YKhWvuscYeQgIbKzrimjV457QIKwR7jFwut7

# Input Data Preprocessing

In [10]:
from bs4 import BeautifulSoup
import re

f = open("""./2018110922gm-00a9-0000-d350bc6b.txt""", 'r', encoding='UTF8')
fTxt = f.read()

#1KyokuSpliter
KyokuSplit = fTxt.split('='*40)
kyoku = 8
MaxKyoku = len(KyokuSplit)-1 #패보 앞에 metadata 뺀 실제 국 수
KyokuSSplit = KyokuSplit[kyoku].split('\n')

#0번 플레이어 버림패
nPlayer = 2
def PlayerKawa(nPlayer):
    Kawa = [line[-3] for line in KyokuSSplit if 'Player '+ str(nPlayer) +': Discard ' in line]
    return Kawa

#print(PlayerKawa(nPlayer))

#N번 플레이어 배패 
KyokuAllHand = KyokuSplit[kyoku].split('Initial Hands:')
InitialHands = KyokuAllHand[1].split('\n')
nPlayerInitialHands = re.findall(r"[^A-Za-z0-9: ]+", InitialHands[nPlayer+1])

Haibo = [line for line in KyokuSSplit if 'Player ' in line]

#N번 플레이어 쯔모패
Draw = []
def PlayerDraw(nPlayer):
    for idx, line in enumerate(KyokuSSplit):
        if 'Player '+ str(nPlayer) +': Draw ' in line:
            Draw.append(line[-3])
        elif 'Player '+ str(nPlayer) + (': Pon ' or ': Chi ') in line:
            #line == KyokuSSplit[idx]
            Draw.append(KyokuSSplit[idx-1][-3])
'''        elif 'Player '+ str(nPlayer) + 'Kan' in line:
            Draw.append(KyokuSSplit[idx+1][-3])'''
    
    return Draw

'''def PlayerDraw(nPlayer):
    for idx, line in enumerate(KyokuSSplit):
        if 'Player '+ str(nPlayer) +': Draw ' in line:
            Draw.append(line[-3])
        elif 'Player '+ str(nPlayer) +': Pon ' in line:
            Draw.append(KyokuSSplit[idx-1][-3])
        elif 'Player '+ str(nPlayer) +': Chi ' in line:
            Draw.append(KyokuSSplit[idx-1][-3])
        #캉 용어 확인 필요    
        elif 'Player '+ str(nPlayer) +': Kan ' in line:
            Draw.append(line[29:30])
    
    return Draw'''


#0번 플레이어 n순 손패 = test data
nPlayerHands = nPlayerInitialHands
#N 순 n turn
TotalTurn = len(PlayerKawa(nPlayer))
for nTurn in range(TotalTurn):
    nPlayerHands.append(PlayerDraw(nPlayer)[nTurn])
    nPlayerHands.sort()
    nPlayerHands.remove(PlayerKawa(nPlayer)[nTurn])

print(nPlayerHands)
#마작 버림패 최대 수 24매

#타패 선택지 모든 패 경우 34장 중 하나로

ValueError: list.remove(x): x not in list

In [ ]:
import tensorflow as tf

# Training Parameters
learning_rate = 0.001
training_steps = 12
batch_size = 1 # 데이터양
display_step = 1

# Network Parameters
num_input = 24*4+14 # 4명의 최대 버림패(24) + 손패 14
timesteps = 24 # 최대 버림패
num_hidden = 128 # hidden layer num of features
num_classes = 14 # 손패 14장

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}
def RNN(x, weights, biases):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = tf.nn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

#with tf.device('/device:GPU:0'):
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True
with tf.Session(config=config) as sess:
    sess.run(init)
    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

'''    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))'''